In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import Model, load_model
from PIL import Image
import random
import time

In [2]:
model = load_model("project.h5")

In [3]:
def winner(user_score,system_score):
    if user_score>system_score:
        return("User Won")
    elif user_score==system_score:
        return( "Tie")
    else:
        return("System Won")
    #pass

In [4]:
# This list will be used to map probabilities to class names, Label names are in alphabatical order.
label_names = [0,1,2,3,4,5,6]
over=[]
attempts = 2
total_attempts=attempts
user_score=0
system_score=0
result=""
cap = cv2.VideoCapture(0)
prob_thrsh=0.7
flag=0
while True:
    _, frame = cap.read()
    # mirror image
    frame = cv2.flip(frame, 1)
           
    # Coordinates of the ROI
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
    #print(frame.shape)
   
    # Drawing the ROI
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    
    # Extracting the ROI
    roi = frame[y1:y2, x1:x2]
    #roi = cv2.resize(roi, (768,768))
    roi=cv2.resize(roi,(128,128))
    # Normalize the image like we did in the preprocessing step, also convert float64 array.
    roi = np.array([roi]).astype('float64') / 255.0
    #roi = cv2.resize(roi,(128,128),Image.ANTIALIAS)
    test=[]
    test.append(roi)
    # Get model's prediction.
    pred = model.predict(test)
    
    # Get the index of the target class.
    target_index = np.argmax(pred[0])

    # Get the probability of the target class
    prob = np.max(pred[0])
    cv2.putText(frame, "prediction: {} {:.2f}%".format(label_names[np.argmax(pred[0])], prob*100 ),
                (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.90, (0, 0, 255), 2, cv2.LINE_AA)
    
    if prob>prob_thrsh:
        time.sleep(2)
        system=random.choice([1,2,3,4,5,6])
        user=label_names[np.argmax(pred[0])]
        if user in over:
            print("Show Different Number")
            cv2.putText(frame, "Show Different Number",
                (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.90, (0, 0, 255), 2, cv2.LINE_AA)
            continue
        else:
            over.append(user)
            print("Adding new score",over)
        print(over)

        user_score=user_score+user
        system_score=system_score+system
        total_attempts -= 1
        #time.sleep(2)
        if system==user:
            print("Same score")
            flag=1
            cv2.putText(frame, "Same score, OUT",
                    (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.90, (0, 0, 255), 2, cv2.LINE_AA)
            break
    cv2.putText(frame, "Your Score: {}  System score: {}".format(user, system),
                (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.90, (0, 0, 255), 2, cv2.LINE_AA)    
    cv2.putText(frame, "Your total score: {} System total score: {}".format(user_score,system_score),
                (10, 360), cv2.FONT_HERSHEY_SIMPLEX, 0.90, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Attempt left:{}".format(total_attempts),
                (10, 330), cv2.FONT_HERSHEY_SIMPLEX, 0.90, (0, 0, 255), 2, cv2.LINE_AA)
    if total_attempts == 0:
        #result=winner(user_score,system_score)
        break
    cv2.imshow("Game", frame)
   
    k = cv2.waitKey(1)
    if k == ord('q'):
        break
if flag==0:
    result=winner(user_score,system_score)
    print("System score: {} user Score: {}".format(system_score,user_score))
    print("Result:",result,"With",(abs(user_score-system_score)),"Runs")
else:
    print("******OUT********")
    print("System Won")
cap.release()
cv2.destroyAllWindows()

Adding new score [6]
[6]
Show Different Number
Show Different Number
Adding new score [6, 2]
[6, 2]
Same score
******OUT********
System Won
